In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

In [5]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [2]:
labels = pd.read_csv('datasets/mikey-devon-labels_validation.csv',delimiter=";")

In [3]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,1,480,University District,Occlusion,47.661045,-122.321037,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:02.905-07,NaN,NaN,f,NaN,NaN,POINT (-122.32104 47.66105)
1,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104)
2,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103)
3,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106)
4,Devon Snyder,6,480,University District,Occlusion,47.660900,-122.321007,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:52.042-07,NaN,NaN,f,NaN,NaN,POINT (-122.32101 47.66090)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-122.26793 47.67204)
2372,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,POINT (-122.26810 47.67193)
2373,mikey,2495,715,Windermere,NoSidewalk,47.672016,-122.268433,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:48.626-07,NaN,3.0,f,street has a sidewalk,NaN,POINT (-122.26843 47.67202)
2374,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,POINT (-122.26818 47.67188)


In [4]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [ ]:
df = cluster_label_type_at_index(label_data,i)[2]
    df["clustered"] = df.duplicated(subset="cluster", keep=False)
    df1 = df[df['clustered'] == True]
    df2 = df1[['username','severity','cluster']]
    d = df2.loc[df2['username']!='mikey']
    m = df2.loc[df2['username'] =='mikey']
    m = m.rename(columns={"severity": "mikey"})
    d = d.rename(columns={"severity": "devon"})
    m = m.drop(columns=['username'])
    d = d.drop(columns=['username'])
    df3 = d.merge(m, how='inner', on='cluster').set_index('cluster')
    return stats.ttest_rel(df3.dropna()['mikey'], df3.dropna()['devon'])

In [12]:
df = cluster_label_type_at_index(chicago,2)[2]
df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
df1 = df[df['clustered'] == True]

In [16]:
df2 = df1[['username','severity','cluster_id']]

In [18]:
d = df2.loc[df2['username']!='mikey']
m = df2.loc[df2['username'] =='mikey']

In [27]:
df3 = pd.merge(d, m, on='cluster_id').set_index('cluster_id')

In [30]:
df3 = df3.rename(columns={"severity_x": "severity_devon", "severity_y": "severity_mikey"})

In [31]:
d_severity = df3["severity_devon"].values
m_severity = df3["severity_mikey"].values

In [33]:
from sklearn.metrics import cohen_kappa_score

In [34]:
#calculate Cohen's Kappa
cohen_kappa_score(d_severity, m_severity )

0.22411278561011183

In [55]:
def severity_irr(label_data,i):
    df = cluster_label_type_at_index(label_data,i)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    df2 = df1[['username','severity','cluster_id']]
    df2 = df2.dropna(subset=['severity'])
    d = df2.loc[df2['username']!='mikey']
    m = df2.loc[df2['username'] =='mikey']
    df3 = pd.merge(d, m, on='cluster_id').set_index('cluster_id')
    df3 = df3.rename(columns={"severity_x": "severity_devon", "severity_y": "severity_mikey"})
    d_severity = df3["severity_devon"].values
    m_severity = df3["severity_mikey"].values
    score = cohen_kappa_score(d_severity, m_severity )
    return score

In [73]:
severity_irr(seattle,5)

0.4915254237288136

In [74]:
def severity_irr_3(label_data,i):
    df = cluster_label_type_at_index(label_data,i)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    df2 = df1[['username','severity','cluster_id']]
    df2 = df2.dropna(subset=['severity'])
    df2['severity'] = df2['severity'].map({1.0: 1.0, 2.0: 1.0,3.0: 2.0, 4.0:3.0, 5.0:3.0})
    d = df2.loc[df2['username']!='mikey']
    m = df2.loc[df2['username'] =='mikey']
    df3 = pd.merge(d, m, on='cluster_id').set_index('cluster_id')
    df3 = df3.rename(columns={"severity_x": "severity_devon", "severity_y": "severity_mikey"})
    d_severity = df3["severity_devon"].values
    m_severity = df3["severity_mikey"].values
    score = cohen_kappa_score(d_severity, m_severity )
    return score

In [75]:
severity_irr_3(seattle,5)

0.6938775510204082

In [94]:
result = pd.DataFrame(columns=['label type','severity levels','kappa'])

In [95]:
for i in [seattle, chicago]:
    for j in [0,2,4,5]: 
        result = result.append({'label type': label_types[j],'severity levels': 5, 'kappa': round(severity_irr(i,j),3)},ignore_index=True)

In [96]:
for i in [seattle, chicago]:
    for j in [0,2,4,5]: 
        result = result.append({'label type': label_types[j],'severity levels': 3, 'kappa': round(severity_irr_3(i,j),3)},ignore_index=True)

In [97]:
cities =['Seattle','Seattle','Seattle','Seattle','Chicago','Chicago','Chicago','Chicago','Seattle','Seattle','Seattle','Seattle','Chicago','Chicago','Chicago','Chicago']
result.insert(loc=0, column='city', value=cities)

In [98]:
result

,city,label type,severity levels,kappa
0,Seattle,CurbRamp,5,0.040
1,Seattle,Problem,5,0.227
2,Seattle,SurfaceProblem,5,0.204
3,Seattle,Obstacle,5,0.492
4,Chicago,CurbRamp,5,-0.039
5,Chicago,Problem,5,0.224
6,Chicago,SurfaceProblem,5,0.211
7,Chicago,Obstacle,5,0.632
8,Seattle,CurbRamp,3,0.066
9,Seattle,Problem,3,0.204


In [99]:
pd.pivot_table(result, values='kappa', index=['city', 'severity levels'],
                    columns=['label type'])

label type               CurbRamp  Obstacle  Problem  SurfaceProblem
city    severity levels                                             
Chicago 3                   0.009     0.788    0.230           0.133
        5                  -0.039     0.632    0.224           0.211
Seattle 3                   0.066     0.694    0.204           0.110
        5                   0.040     0.492    0.227           0.204